In [263]:
using JuMP
using Gurobi
using MultilinearOpt
using Plots
gr()

Plots.GRBackend()

In [348]:
m = Model(solver=GurobiSolver())
N = 5
f_grav = -9.81
Δt_min = 0.05
Δt_max = 0.5
# f_grav = 0
@variable m x[1:N] >= 0
@variable m -10 <= v[1:N] <= 10
@variable m 0 <= f[1:N-1] <= 100
@variable m Δt_min <= Δt[1:N-1] <= Δt_max
@variable m Δt_min <= Δt_aux[1:N-1] <= Δt_max
@constraint m Δt .== Δt_aux
@variable m Δt_min^2 <= Δt²[1:N-1] <= Δt_max^2
@variable m contact[1:N-1] Bin
@constraint m [i=1:N-1] Δt²[i] == Δt[i] * Δt_aux[i]

Δx = diff(x)
Δv = diff(v)
@constraint m [i=1:N-1] Δv[i] == Δt[i] * f[i] + f_grav * Δt[i]
@constraint m [i=1:N-1] Δx[i] == v[i] * Δt[i] + 0.5 * Δt²[i] * f[i] + 0.5 * Δt²[i] * f_grav

@constraint m [i=1:N-1] f[i] <= contact[i] * 20
@constraint m [i=1:N-1] x[i] <= (1 - contact[i]) * 10

@constraint m x[1] == 1
@constraint m v[1] == 0
@constraint m v[end] == 0

MultilinearOpt.relaxbilinear!(m, method=:MisenerLog1)
@objective m Min 0.001 * sum(f.^2) + 1000 * x[end]^2 + v[end]^2

solve(m)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 379 rows, 350 columns and 2063 nonzeros
Model has 6 quadratic objective terms
Variable types: 298 continuous, 52 integer (52 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-03, 2e+03]
  Bounds range     [3e-03, 1e+02]
  RHS range        [3e-03, 1e+01]
Presolve removed 71 rows and 70 columns
Presolve time: 0.00s
Presolved: 308 rows, 280 columns, 1634 nonzeros
Presolved model has 4 quadratic objective terms
Variable types: 233 continuous, 47 integer (47 binary)

Root relaxation: objective 1.123555e-02, 302 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.01124    0   28          -    0.01124      -     -    0s
     0     0    0.01196    0   36          -    0.01196      - 

:Optimal

In [349]:
t = vcat(0, cumsum(getvalue(Δt)))
plt = plot(t, getvalue(x), markershape=:circle)
tt = linspace(t[1], t[end])
plot!(plt, tt, 1 + 0.5 * f_grav * tt.^2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 37.9699,14.7204 49.0278,14.8702 60.0858,15.3193 71.1437,16.0678 82.2016,17.1158 93.2595,18.4631 104.317,20.1099 115.375,22.0561 126.433,24.3017 137.491,26.8468 
 148.549,29.6912 159.607,32.8351 170.665,36.2783 181.723,40.021 192.781,44.0631 203.839,48.4047 214.897,53.0456 225.955,57.9859 237.012,63.2257 248.07,68.7649 
 259.128,74.6035 270.186,80.7415 281.244,87.1789 292.302,93.9158 303.36,100.952 314.418,108.288 325.476,115.923 336.534,123.857 347.592,132.091 358.65,140.625 
 369.707,149.457 380.765,158.589 391.823,168.021 402.881,177.752 413.939,187.782 424.997,198.112 436.055,208.742 447.113,219.67 458.171,230.898 469.229,242.426 
 480.287,254.253 491.345,266.379 502.402,278.805 513.46,291.53 524.518,304.554 535.576,317.878 546.634,331.502 557.692,345.425 568.75,359.647 579.808,374.168 
 
 "/>
 
 
 
 
 
 
 
 
 y1 
 
 
 
 y2

In [350]:
plot(t[1:end-1], getvalue(f), markershape=:circle)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1